In [ ]:
import torch
import csv
!pip install wandb
import wandb
import heapq
import torch.nn as nn
import random
import numpy as np
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
import matplotlib.ticker as ticker
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
wandb.login(key='b032cc059132c9aac4f1b317f6f9ad007ef9e4d4')
wandb.init(project="Assignment3_final")

wandb: Currently logged in as: cs22m083. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/aksharantar_dataset/tel/'

/content/drive/MyDrive/aksharantar_dataset/tel


In [ ]:
tsv_file = open("/content/drive/MyDrive/aksharantar_dataset/tel/tel_train.csv")
test_tsv_file = open("/content/drive/MyDrive/aksharantar_dataset/tel/tel_test.csv")
val_tsv_file = open("/content/drive/MyDrive/aksharantar_dataset/tel/tel_valid.csv")
read_tsv = csv.reader(tsv_file)
test_read_tsv = csv.reader(test_tsv_file)
val_read_tsv = csv.reader(val_tsv_file)

In [ ]:
train_X, train_Y = [], []
for i in read_tsv:
    train_X.append(i[0])
    train_Y.append(i[1])
    
test_X,test_Y = [], []
for i in test_read_tsv:
    test_X.append(i[0])
    test_Y.append(i[1])
    
val_X, val_Y = [], []
for i in val_read_tsv:
    val_X.append(i[0])
    val_Y.append(i[1])
    


test_X, test_Y = np.array(test_X), np.array(test_Y)
t_range = test_Y.shape[0]
val_X, val_Y = np.array(val_X),  np.array(val_Y)
v_range = val_Y.shape[0]
train_X, train_Y = np.array(train_X), np.array(train_Y)
tr_range = train_Y.shape[0]
for i in range(t_range):
    test_Y[i] = "\t" + test_Y[i] + "\n"
    
for i in range(v_range):
    val_Y[i] = "\t" + val_Y[i] + "\n"

for i in range(tr_range):
    train_Y[i] = "\t" + train_Y[i] + "\n"
ch_end = " "

In [ ]:
val_input_corpus, output_corpus, input_corpus,  val_output_corpus = set(), set(), set(), set()

for word in train_Y:
    for char in word:
        if char not in output_corpus:
            output_corpus.add(char)

for word in train_X:
    for char in word:
        if char not in input_corpus:
            input_corpus.add(char)

output_corpus.add(ch_end)
input_corpus.add(ch_end)


for word in val_Y:
    for char in word:
        if char not in val_output_corpus:
            val_output_corpus.add(char)

output_corpus,  input_corpus = sorted(list(output_corpus)), sorted(list(input_corpus))
num_encoder_tokens =  len(input_corpus)  

for word in val_X:
    for char in word:
        if char not in val_input_corpus:
            val_input_corpus.add(char)

num_decoder_tokens = len(output_corpus)


In [ ]:
max_decoder_seq_length,max_encoder_seq_length = max([len(txt) for txt in train_Y]), max([len(txt) for txt in train_X]) + 2

In [ ]:
print("Number of unique input tokens:", num_encoder_tokens)
print("Max sequence length for outputs:", max_decoder_seq_length)
print("Number of samples:", len(train_X))
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Number of unique output tokens:", num_decoder_tokens)

Number of unique input tokens: 27
Max sequence length for outputs: 21
Number of samples: 51200
Max sequence length for inputs: 30
Number of unique output tokens: 65


In [ ]:
batch_size, num_epochs, learning_rate = 32, 10, 0.001
load_model, training = False, False
input_size_encoder, input_size_decoder = num_encoder_tokens, num_decoder_tokens
output_size = num_decoder_tokens
encoder_embedding_size, decoder_embedding_size = 256, 256
hidden_size = 256  # Needs to be the same for both RNN's
num_enc_layers, num_dec_layers = 1, 1
enc_dropout, dec_dropout = 0.1, 0.1
d_type, td_type = "int64", torch.int64

In [ ]:
train_ln = len(train_X)
val_ln = len(val_X)
input_char_index, output_char_index = dict([(char, i) for i, char in enumerate(input_corpus)]), dict([(char, i) for i, char in enumerate(output_corpus)])
input_data, target_data = np.zeros((max_encoder_seq_length,train_ln), dtype= d_type), np.zeros((max_decoder_seq_length,train_ln), dtype=d_type)
c_end = " "
input_data_val, target_data_val = np.zeros((max_encoder_seq_length,val_ln), dtype=d_type), np.zeros((max_decoder_seq_length,val_ln), dtype=d_type)
for i, (x, y) in enumerate(zip(val_X, val_Y)):
    t_n = i+ val_ln
    for t, char in enumerate(y):
        target_data_val[t, i] = output_char_index[char]
    t_n = t+1       
    target_data_val[t_n :,i] = output_char_index[c_end]
    t_n = 0
    for t, char in enumerate(x):
        input_data_val[t, i] = input_char_index[char]
    t_n = t+1   
    input_data_val[t_n :,i] = input_char_index[c_end] 
t_n =0    
for i, (x, y) in enumerate(zip(train_X, train_Y)):
    t_n = i+ val_ln
    for t, char in enumerate(y):
        target_data[t, i] = output_char_index[char]
    t_n  = t+1        
    target_data[t_n :,i] = output_char_index[c_end]
    t_n =0
    for t, char in enumerate(x):
        input_data[t, i] = input_char_index[char]
    t_n = t+1   
    input_data[t_n :,i] = input_char_index[c_end]

In [ ]:
# convertin numpy arrays to tensors
input_data_val, target_data_val = torch.tensor(input_data_val,dtype = td_type), torch.tensor(target_data_val,dtype = td_type)
input_data, target_data = torch.tensor(input_data,dtype = td_type), torch.tensor(target_data,dtype = td_type)

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_target_char_index,reverse_input_char_index, t_z = dict((i, char) for char, i in output_char_index.items()), dict((i, char) for char, i in input_char_index.items()), 0

**LSTM** 


In [ ]:
class Encoder(nn.Module): 
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        drop_par = dropout
        self.dropout = nn.Dropout(drop_par)
        self.num_layers, self.hidden_size, = num_layers, hidden_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)

    def forward(self, x): # x shape: (seq_length, N) where N is batch size
        drop_par = self.embedding(x)
        embedding = self.dropout(drop_par) # embedding shape: (seq_length, N, embedding_size)
        outputs, (hidden, cell) = self.rnn(self.dropout(drop_par)) # outputs shape: (seq_length, N, hidden_size)
        return hidden, cell

class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder):

        super(Seq2Seq, self).__init__()  
        self.decoder, self.encoder = decoder, encoder

    def forward(self, source, target, teacher_force_ratio=0.2):
        batch_size, target_len, target_vocab_size = source.shape[1], target.shape[0], num_decoder_tokens

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        # Grab the first input to the Decoder which will be <SOS> token
        x = target[0]
        hidden, cell = self.encoder(source)

        for t in range(1, target_len):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, cell = self.decoder(x, hidden, cell)

            # Store next output prediction
            outputs[t], best_guess = output, output.argmax(1)
            # Get the best word the Decoder predicted (index in the vocabulary)
            x = best_guess if random.random() >= teacher_force_ratio else target[t]

        return outputs

class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout):
        super(Decoder, self).__init__()
        drop_par, hidden_layer_size = dropout, hidden_size
        self.dropout,self.num_layers, self.hidden_size = nn.Dropout(drop_par),  num_layers, hidden_layer_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout= drop_par)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell): # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x  = x.unsqueeze(0) 
        drop_par = self.embedding(x)
        embedding = self.dropout(drop_par) # embedding shape: (1, N, embedding_size)
        outputs, (hidden, cell) = self.rnn(self.dropout(drop_par), (hidden, cell)) # outputs shape: (1, N, hidden_size)
        #predictions = self.fc(outputs)
        
        
        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        predictions = self.fc(outputs).squeeze(0)

        return predictions, hidden, cell


**GRU**

In [ ]:
class EncoderGRU(nn.Module): 
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(EncoderGRU, self).__init__()
        drop_par = p
        self.dropout = nn.Dropout(drop_par)
        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout= drop_par)

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size
        drop_par = self.embedding(x)
        embedding = self.dropout(drop_par) # embedding shape: (seq_length, N, embedding_size)
        outputs, hidden = self.rnn(self.dropout(drop_par)) # outputs shape: (seq_length, N, hidden_size)
        return hidden

class Seq2SeqGR(nn.Module):
    
    def __init__(self, encoder, decoder):
        
        super(Seq2SeqGR, self).__init__()
        self.decoder, self.encoder  = decoder, encoder

    def forward(self, source, target, teacher_force_ratio=0.2):
        batch_size, target_len, target_vocab_size = source.shape[1], target.shape[0], num_decoder_tokens
        x = target[0]  # Grab the first input to the Decoder which will be <SOS> token
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        hidden= self.encoder(source)
        
        for t in range(1, target_len):
            # Use previous hidden, cell as context from encoder at start
            output, hidden = self.decoder(x, hidden)

            # Store next output prediction
            # Get the best word the Decoder predicted (index in the vocabulary)
            outputs[t], best_guess = output, output.argmax(1)
            
            x = best_guess if random.random() >= teacher_force_ratio else target[t]

        return outputs

class DecoderGRU(nn.Module):
    def __init__(
        self, input_size, embedding_size, hidden_size, output_size, num_layers, p):
        super(DecoderGRU, self).__init__()
        drop_par , hidden_layer_size = p, hidden_size
        self.dropout, self.num_layers, self.hidden_size = nn.Dropout(drop_par), num_layers, hidden_layer_size

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout= drop_par)
        self.fc = nn.Linear(hidden_size, output_size)


    def forward(self, x, hidden):
        x = x.unsqueeze(0) # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        drop_par = self.embedding(x)
        embedding = self.dropout(drop_par) # embedding shape: (1, N, embedding_size)
        outputs, hidden = self.rnn(self.dropout(drop_par), hidden) # outputs shape: (1, N, hidden_size)
        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        # predictions = self.softmax(predictions)
        predictions = self.fc(outputs).squeeze(0)
        return predictions, hidden


**RNN**

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(
        self, input_size, embedding_size, hidden_size, output_size, num_layers, p):
        super(DecoderRNN, self).__init__()
        drop_par, hidden_layer_size = p , hidden_size
        self.dropout, self.num_layers, self.hidden_size = nn.Dropout(p), num_layers, hidden_layer_size 
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout= drop_par)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = x.unsqueeze(0)# x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        drop_par = self.embedding(x)
        embedding = self.dropout(drop_par)
        # embedding shape: (1, N, embedding_size)

        outputs, hidden = self.rnn(self.dropout(drop_par), hidden)
        #predictions = self.fc(outputs)
        predictions = self.fc(outputs).squeeze(0)

        return predictions, hidden

class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(EncoderRNN, self).__init__()
        drop_par ,hidden_layer_size = p,hidden_size
        self.dropout, self.num_layers, self.hidden_size = nn.Dropout(drop_par), num_layers, hidden_layer_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout= drop_par)

    def forward(self, x):# x shape: (seq_length, N) where N is batch size
        drop_par = self.embedding(x)
        embedding = self.dropout(drop_par) # embedding shape: (seq_length, N, embedding_size)
        outputs, hidden = self.rnn(self.dropout(drop_par))
        # outputs shape: (seq_length, N, hidden_size)
        # hidden shape: (num_layers, N, hidden_size)
        return hidden

In [ ]:
def translate(model, word, input_char_index, output_char_index, reverse_input_char_index, 
              reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length, 
              num_encoder_tokens, num_decoder_tokens, device):
    
    data, word_t = np.zeros((max_encoder_seq_length,1), dtype= d_type), ''
    t_z = 0
    for t, char in enumerate(word):
        data[t, 0] = input_char_index[char]
    t_z = t+1   
    data[t_z :,0] = input_char_index[c_end]
    
    data = torch.tensor(data,dtype = td_type).to(device)

    with torch.no_grad():
        hidden, cell = model.encoder(data)
    out_t = output_char_index['\t']    
    out_chr_reshape = np.array(out_t).reshape(1,)    
    x = torch.tensor(out_chr_reshape).to(device)

    for t in range(1, max_decoder_seq_length):
        
        output, hidden, cell = model.decoder(x, hidden, cell)
        # best_guess = output.argmax(1)
        ch = reverse_target_char_index[output.argmax(1).item()]
        
        if ch != '\n':
            word_t = word_t+ch
        else:
            break

    return word_t

In [ ]:
def translateGR(model, word, input_char_index, output_char_index, reverse_input_char_index, 
              reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length, 
              num_encoder_tokens, num_decoder_tokens, device):
    
    data, word_t = np.zeros((max_encoder_seq_length,1), dtype= d_type), ''
    t_z = 0
    for t, char in enumerate(word):
        data[t, 0] = input_char_index[char]
    t_z = t+1   
    data[t_z :,0] = input_char_index[c_end]
    
    data = torch.tensor(data,dtype = td_type).to(device)

    with torch.no_grad():
        hidden = model.encoder(data)
    out_t = output_char_index['\t']
    out_reshape = np.array(out_t).reshape(1,)
    x = torch.tensor(out_reshape).to(device)
    
    for t in range(1, max_decoder_seq_length):
        output, hidden = model.decoder(x, hidden)
        #best_guess = output.argmax(1)
        
        ch = reverse_target_char_index[output.argmax(1).item()]
        if ch != '\n':
            word_t = word_t+ch
        else:
            break

    return word_t

In [ ]:
def beam_search(model, word, input_char_index, output_char_index, reverse_input_char_index,reverse_target_char_index, max_encoder_seq_length, 
                max_decoder_seq_length,num_encoder_tokens, num_decoder_tokens, beam_width, device, length_penalty=0.6):


    # Encode the input word
    data, word_t = np.zeros((max_encoder_seq_length, 1), dtype= d_type), ''
    t_z , bw = 0, beam_width
    for t, char in enumerate(word):
        data[t, 0] = input_char_index[char]
    t_z = t+1
    data[t_z:, 0] = input_char_index[c_end]

    data = torch.tensor(data, dtype= td_type ).to(device)

    with torch.no_grad():
        hidden,cell = model.encoder(data)

    # Initialize beam
    out_t = output_char_index['\t']
    out_reshape = np.array(out_t).reshape(1,)
    hidden_par = hidden.unsqueeze(0)
    initial_sequence = torch.tensor(out_reshape).to(device)
    beam = [(0.0, initial_sequence, hidden_par)]  # [(score, sequence, hidden)]

    for _ in range(max_decoder_seq_length):
      candidates = []
      for score, seq, hidden in beam:
          # last_token = seq[-1].item()
          if seq[-1].item() == output_char_index['\n']:
              # If the sequence ends with the end token, add it to the candidates
              candidates.append((score, seq, hidden))
              continue
          lt_reshape = np.array(seq[-1].item()).reshape(1,) # last_token = seq[-1].item()
          hidden_upar = hidden.squeeze(0)
          x = torch.tensor(lt_reshape).to(device)
          output, hidden,cell = model.decoder(x, hidden_upar,cell)
          probabilities = F.softmax(output, dim=1)
          cal_cand = 0
          # Get the top-k probabilities and tokens
          topk_probs, topk_tokens = torch.topk(probabilities, k=bw)

          for prob, token in zip(topk_probs[0], topk_tokens[0]):
              cal_cand = prob
              n_hidden = hidden.clone()
              new_seq = torch.cat((seq, token.unsqueeze(0)), dim=0)
              ln_ns = len(new_seq)
              ln_pf = ((ln_ns - 1) / 5)
              new_hidden = n_hidden.unsqueeze(0)
              #length_penalty_factor = ln_pf ** length_penalty  # Adjust penalty factor as needed
              candidate_cal = score + torch.log(prob).item() / (ln_pf ** length_penalty)
              candidates.append((candidate_cal, new_seq, new_hidden))

      beam = heapq.nlargest(bw, candidates, key=lambda x: x[0])# Select top-k candidates based on the accumulated scores

      
    best_score, best_sequence, _ = max(beam, key=lambda x: x[0]) # Select the best sequence from the beam as the output
    
    cal_score = best_score
    word_t = ''.join([reverse_target_char_index[token.item()] for token in best_sequence[1:-1]])
    cal_score = 0
    return word_t

In [ ]:
def beam_searchGR(model, word, input_char_index, output_char_index, reverse_input_char_index,
                reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length,
                num_encoder_tokens, num_decoder_tokens, beam_width, device, length_penalty=0.6):

    # Encode the input word
    data, word_t = np.zeros((max_encoder_seq_length, 1), dtype= d_type), ''
    t_z, tab_pad, nl_pad,prob_dim, bw = 0, '\t', '\n', 1, beam_width
    for t, char in enumerate(word):
        data[t, 0] = input_char_index[char]
    t_z = t+1
    data[t_z :, 0] = input_char_index[c_end]

    data = torch.tensor(data, dtype= td_type).to(device)

    with torch.no_grad():
        hidden = model.encoder(data)

    # Initialize beam
    out_t = output_char_index[tab_pad]
    out_reshape = np.array(out_t).reshape(1,)
    hidden_par = hidden.unsqueeze(0)
    initial_sequence = torch.tensor(out_reshape).to(device)
    beam = [(0.0, initial_sequence, hidden_par)]  # [(score, sequence, hidden)]

    for _ in range(max_decoder_seq_length):
      score_cal , candidates = 0, []
      for score, seq, hidden in beam:
          score_cal = score_cal + score
          last_token = seq[-1].item()
          if last_token == output_char_index[nl_pad]:
              # If the sequence ends with the end token, add it to the candidates
              candidates.append((score, seq, hidden))
              
              continue
          
          last_t_reshape = np.array(last_token).reshape(1,)
          hidden_par = hidden.squeeze(0)
          x = torch.tensor(last_t_reshape).to(device)
          output, hidden = model.decoder(x, hidden_par)
          probabilities = F.softmax(output, dim = prob_dim)

          # Get the top-k probabilities and tokens
          topk_probs, topk_tokens = torch.topk(probabilities, k= bw)
          cal_score = score
          for prob, token in zip(topk_probs[0], topk_tokens[0]):
              cal_score = cal_score + prob
              new_seq = torch.cat((seq, token.unsqueeze(0)), dim=0)
              n_hidden = hidden.clone()
              new_seq_ln  = len(new_seq)
              pen_fac = (new_seq_ln - 1) / 5
              new_hidden = n_hidden.unsqueeze(0)
              #length_penalty_factor = ((pen_fac) ** length_penalty)  # Adjust penalty factor as needed
              
              candidates.append((score + torch.log(prob).item() / ((pen_fac) ** length_penalty), new_seq, new_hidden))
              cal_score = 0
      # Select top-k candidates based on the accumulated scores
      beam = heapq.nlargest(beam_width, candidates, key = lambda x: x[0])

    
    best_score, best_sequence, _ = max(beam, key=lambda x: x[0])# Select the best sequence from the beam as the output
    
    word_t = ''.join([reverse_target_char_index[token.item()] for token in best_sequence[1:-1]])

    return word_t

In [ ]:
num_epochs, batch_size, learning_rate = 2, 32, 0.001
load_model, training= False, False
input_size_encoder, input_size_decoder = num_encoder_tokens, num_decoder_tokens
hidden_size = 256  # Needs to be the same for both RNN's
output_size = num_decoder_tokens
dec_dropout, enc_dropout = 0.1, 0.1
encoder_embedding_size, decoder_embedding_size = 256, 256
num_dec_layers,num_enc_layers = 2, 2

In [ ]:
def training(num_encoder_tokens,input_embedding_size, dp, cell_type, hidden_size, num_enc_layers, num_dec_layers,num_epochs,output_size,input_size_decoder,batch_size,beam_width):
    #Decoder Selection
    if(cell_type=="LSTM"):
        decoder_net = Decoder(input_size_decoder,input_embedding_size,hidden_size,output_size,num_dec_layers,dp).to(device)
    elif(cell_type=="GRU"):
        decoder_net = DecoderGRU(input_size_decoder,decoder_embedding_size,hidden_size,output_size,num_dec_layers,dec_dropout).to(device)
    else:
        decoder_net = DecoderRNN(input_size_decoder,decoder_embedding_size,hidden_size,output_size,num_dec_layers,dec_dropout).to(device)
    #Encoder Selection
    if(cell_type=="LSTM"):
        encoder_net = Encoder(input_size_encoder,input_embedding_size, hidden_size, num_enc_layers,dp).to(device)
    elif(cell_type=="GRU"):
        encoder_net = EncoderGRU(input_size_encoder, encoder_embedding_size, hidden_size, num_enc_layers, enc_dropout).to(device)
    else:
        encoder_net = EncoderRNN(input_size_encoder, encoder_embedding_size, hidden_size, num_enc_layers, enc_dropout).to(device)
    
    #Model Selection
    if(cell_type=="LSTM"):    
        model = Seq2Seq(encoder_net, decoder_net).to(device)
    else:
        model = Seq2SeqGR(encoder_net, decoder_net).to(device)
    
    split_dim, bs,  m_norm = 1, batch_size, 1
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    train_ds_y, train_ds_x = torch.split(target_data, bs, dim = split_dim), torch.split(input_data, bs, dim = split_dim)
    correct_prediction  = 0
    #print(train_ds_x)
    for epoch in range(num_epochs):
        print(f"[Epoch {epoch} / {num_epochs}]")
        model.eval()
        total_count = len(val_X)
        model.train()
        for i, (x,y) in enumerate(zip(train_ds_x,train_ds_y)):
            # Get input and targets and get to cuda
            target, inp_data = y.to(device), x.to(device)
            # Forward prop
            output = model(inp_data, target)
            target, output = target[1:].reshape(-1),  output[1:].reshape(-1, output.shape[2])
            optimizer.zero_grad()
            loss = criterion(output, target)
            loss.backward() # Back prop
            # Clip to avoid exploding gradient issues, makes sure grads are
            # within a healthy range
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = m_norm)
            # Gradient descent step
            optimizer.step()
        correct_pred, total_words, lstm_bw = 0, total_count, 1
        model.eval()
        for i in range(total_count):
            if(cell_type=="LSTM"):
                decoded_sentence = beam_search(model,val_X[i], input_char_index, output_char_index, reverse_input_char_index, reverse_target_char_index, 
                                                     max_encoder_seq_length, max_decoder_seq_length,num_encoder_tokens, num_decoder_tokens,lstm_bw,device)
            else:
                 decoded_sentence = beam_searchGR(model,val_X[i], input_char_index, output_char_index, reverse_input_char_index, reverse_target_char_index, 
                                                        max_encoder_seq_length, max_decoder_seq_length,num_encoder_tokens, num_decoder_tokens,beam_width,device)
                 
            if decoded_sentence == val_Y[i][1:-1]:
                correct_pred = correct_pred + 1
        test_accuracy = correct_pred / total_words
        print(correct_pred / total_words)  # test_accuracy = correct_pred / total_words
        wandb.log({'Val_accuracy' : test_accuracy*100})

In [ ]:
sweep_config = {
    'method': 'bayes',
    'parameters': {
                   'dec_num_layers':{
                      'values': [1,2,3] 
                      },
                   'embedding_size': {
                      'values': [128, 256, 512] 
                      },
                   'hidden_size': {
                      'values': [128, 256, 512]
                      },
                   'dropout': {
                      'values': [0.1, 0.2, 0.3, 0.4]
                      },
                   'cell_type': {
                      'values': ['RNN','LSTM','GRU']
                      },
                   'beam_search':{
                      'values':[1,2,3,4,5]
                      },
                   'epochs' :{
                      'values':[10,20,30,40]
                      },
                   'enc_num_layers': {
                      'values': [1,2,3]
                      },
                   'batch_size': {
                      'values': [128,256,512]
                      }
                },
    'metric': {'goal': 'maximize', 'name': 'val_accuracy'}
}

In [ ]:
def train():
    var1 = wandb.init(project="Assignment3_final")
    var2 = var1.config
    # var2 is a variable that holds and saves hyperparameters and inputs
    wandb.run.name = 'Cell_Type:-' + var2.cell_type + ', Batch_Size:-' + str(var2.batch_size) + ', Epochs:-' + str(var2.epochs) + ', Dropout:-' + str(var2.dropout) + ', Beam_Search:-' + str(var2.beam_search) +', Embedding_Size:-' + str(var2.embedding_size) + ', Hidden_Size:-' + str(var2.hidden_size) + ', Enc_Num_Layers:-' + str(var2.enc_num_layers) + ', Dec_Num_Layers:-' + str(var2.dec_num_layers)
    if(var2.cell_type == 'RNN'):
        epochs = 10
        training(input_size_encoder ,var2.embedding_size, var2.dropout, var2.cell_type, var2.hidden_size, var2.enc_num_layers,  var2.enc_num_layers,epochs,num_decoder_tokens,num_decoder_tokens,var2.batch_size,var2.beam_search)
    else:
        training(input_size_encoder ,var2.embedding_size, var2.dropout, var2.cell_type, var2.hidden_size, var2.enc_num_layers,  var2.enc_num_layers,var2.epochs,num_decoder_tokens,num_decoder_tokens,var2.batch_size,var2.beam_search)


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Assignment3_final")
wandb.agent(sweep_id, train, count=40)